<a href="https://colab.research.google.com/github/PvPaulinho/Deep-Learning-Facul-/blob/main/Tarefa9_IdentificacaoPoliticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 9 - Identificação facial 

Uma agremiação brasileira resolveu usar um método de identificação facial para autenticar o cadastro de seus filiados. Infelizmente, a solução foi pouco testada e foi um gargalo no processamento dos cadastros. 

Nessa tarefa, vamos testar os métodos de reconhecimento facial com políticos filiados à referida agremiação. 

Para completar a tarefa, vamos também testar a rede treinada com fotos dos mesmos políticos com máscaras faciais e avaliar acurácia nas predições com as redes originalmente treinadas. 

O dataset foi disponibilizado no github com os conjuntos de treinamento e validação separados. Um segundo conjunto de validação, com máscaras também está disponível no repositório. 

Abaixo, a seqüência para fazer download do dataset e a extração das faces. 

Você deverá: 

1. Escolher um método para implementar a identificação (embedding+SVM ou transfer learning com fine tuning, conforme as lições anteriores) 

2. Treinar a rede, avaliar acurácia de treinamento e validação

3. Avaliar a acurácia para o conjunto das imagens com máscara facial

## Alguns ajustes iniciais

In [1]:
!pip install mtcnn
!pip install git+https://github.com/jsansao/keras-vggface.git
!pip install keras_applications


     |████████████████████████████████| 2.3 MB 5.5 MB/s 
  Cloning https://github.com/jsansao/keras-vggface.git to /tmp/pip-req-build-ubd20vvw
  Running command git clone -q https://github.com/jsansao/keras-vggface.git /tmp/pip-req-build-ubd20vvw
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8315 sha256=88067acbe33b197c22bf1af7fd71c6f5a83947b33a18e488bc8f34191ac8cca7
  Stored in directory: /tmp/pip-ephem-wheel-cache-klguc22c/wheels/17/1d/de/3357313e72504b373dcd31226714c9477da009000aeb307586
Successfully built keras-vggface
     |████████████████████████████████| 50 kB 3.0 MB/s 


In [2]:
!wget https://github.com/jsansao/TucanoDataset/archive/refs/heads/main.zip
!unzip main.zip

--2021-12-17 01:47:19--  https://github.com/jsansao/TucanoDataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/jsansao/TucanoDataset/zip/refs/heads/main [following]
--2021-12-17 01:47:19--  https://codeload.github.com/jsansao/TucanoDataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [            <=>     ]  74.78M  20.5MB/s    in 3.7s    

2021-12-17 01:47:23 (20.5 MB/s) - ‘main.zip’ saved [78409321]

Archive:  main.zip
e10dc72508ba64b2c5341f1248e1c94089057ba9
   creating: TucanoDataset-main/
   creating: TucanoDataset-main/masc_val/
   c

## Funções para extração das faces e geração do dataset com faces

In [3]:
# face detection 
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
import mtcnn

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = mtcnn.MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path, (224,224))
		# store
		faces.append(face)
	return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)



## Geração dos arquivos com as faces e rótulos

In [4]:
# load train dataset
trainX, trainy = load_dataset('/content/TucanoDataset-main/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('/content/TucanoDataset-main/val/')
# load mask dataset
valX, valy = load_dataset('/content/TucanoDataset-main/masc_val/')

# save arrays to one file in compressed format
savez_compressed('tucano-faces-dataset.npz', trainX, trainy, testX, testy, valX, valy)

>loaded 15 examples for class: JoaoDoria
>loaded 15 examples for class: AecioNeves
>loaded 15 examples for class: EduardoLeite
>loaded 15 examples for class: GeraldoAlckmin
>loaded 15 examples for class: JoseSerra
>loaded 15 examples for class: ArthurVirgilio
>loaded 15 examples for class: JoseAnibal
>loaded 15 examples for class: FHC
(120, 224, 224, 3) (120,)
>loaded 11 examples for class: JoaoDoria
>loaded 15 examples for class: AecioNeves
>loaded 8 examples for class: EduardoLeite
>loaded 6 examples for class: GeraldoAlckmin
>loaded 14 examples for class: JoseSerra
>loaded 11 examples for class: ArthurVirgilio
>loaded 9 examples for class: JoseAnibal
>loaded 4 examples for class: FHC
>loaded 6 examples for class: JoaoDoria
>loaded 8 examples for class: AecioNeves
>loaded 7 examples for class: EduardoLeite
>loaded 5 examples for class: GeraldoAlckmin
>loaded 1 examples for class: JoseSerra
>loaded 6 examples for class: ArthurVirgilio
>loaded 5 examples for class: JoseAnibal
>loaded 4